In [3]:
import numpy as np
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers
import cv2
import zipfile
import shutil
import random
import pandas as pd
import csv
import os

In [3]:
import kagglehub

kagglehub.login()

# Replace with path to directory containing model files.
LOCAL_MODEL_DIR = 'path/to/files'

MODEL_SLUG = 'my_model' # Replace with model slug.

# Learn more about naming model variations at
# https://www.kaggle.com/docs/models#name-model.
VARIATION_SLUG = 'default' # Replace with variation slug.

kagglehub.model_upload(
  handle = f"kartikgarg74/{MODEL_SLUG}/keras/{VARIATION_SLUG}",
  local_model_dir = LOCAL_MODEL_DIR,
  version_notes = 'Update 2024-08-01')

Uploading Model https://www.kaggle.com/models/kartikgarg74/my_model/keras/default ...
Model 'my_model' does not exist or access is forbidden for user 'kartikgarg74'. Creating or handling Model...


HTTPError: 401 Client Error: Unauthorized for url: https://www.kaggle.com/api/v1/models/create/new

In [4]:
import tensorflow as tf

# Initialize the GPU for Mac M2
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Set memory growth to avoid allocation errors
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(f'{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs')
    except RuntimeError as e:
        print(e)
else:
    print("No GPUs found.")


No GPUs found.


In [6]:
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Number of devices: 1


In [5]:
data_path="/Users/raghavgarg/Downloads/archive/"

In [6]:
max_images_per_class = 1000
classes = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']
def collect_data(folder):
    data = []
    for clas in classes:
        class_path = os.path.join(data_path, folder, clas)
        files = [os.path.join(folder, clas, file) for file in os.listdir(class_path) if file.endswith('.jpg')]
        if len(files) > max_images_per_class:
            files = random.sample(files, max_images_per_class)
        data.extend([[file, clas] for file in files])
    return data
max_images_per_class_1 = 100
def collect_data_1(folder):
    data = []
    for clas in classes:
        class_path = os.path.join(data_path, folder, clas)
        files = [os.path.join(folder, clas, file) for file in os.listdir(class_path) if file.endswith('.jpg')]
        if len(files) > max_images_per_class_1:
            files = random.sample(files, max_images_per_class_1)
        data.extend([[file, clas] for file in files])
    return data
# Collect train and test data
train_data = collect_data('seg_train/seg_train')
test_data = collect_data_1('seg_test/seg_test')

In [7]:
# Write train and test CSV files
os.makedirs('data', exist_ok=True)
with open('data/train.csv', 'w', newline='') as f:
    csvw = csv.writer(f)
    csvw.writerow(['filename', 'class'])
    for item in train_data:
        csvw.writerow(item)

with open('data/test.csv', 'w', newline='') as f:
    csvw = csv.writer(f)
    csvw.writerow(['filename', 'class'])
    for item in test_data:
        csvw.writerow(item)

# Load data into dataframes
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

In [8]:
# Set data augmentation techniques
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.2,
    rotation_range=360,
    width_shift_range=0.1,
    height_shift_range=0.1,
    channel_shift_range=50,
    brightness_range=(0, 1.2),
    preprocessing_function=keras.applications.imagenet_utils.preprocess_input
)

test_datagen = keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=keras.applications.imagenet_utils.preprocess_input
)

In [17]:
# Create data generators
batch_size = 70
with strategy.scope():
    train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=data_path,
        x_col="filename",
        y_col="class",
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True
    )

    validation_generator = test_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory=data_path,
        x_col="filename",
        y_col="class",
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True
    )

train_img_num = len(train_generator.filenames)

Found 6000 validated image filenames belonging to 6 classes.
Found 600 validated image filenames belonging to 6 classes.


In [ ]:
name = "Intel-DenseNet169-depthw-constraints-224"
os.makedirs("models", exist_ok=True)  # Create folder for saving checkpoints
os.makedirs("reports", exist_ok=True)  # Create folder for saving evaluation reports
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

keras.backend.clear_session()  # Clear backend
shape = (224, 224, 3)
input_tensor = keras.Input(shape=shape)
with strategy.scope():
    base_model = keras.applications.DenseNet169(input_tensor=input_tensor, weights=None, include_top=False)
    depth = keras.layers.DepthwiseConv2D(
        tuple(base_model.output.shape[1:3]),
        depthwise_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.01),
        bias_initializer=keras.initializers.Zeros(),
        depthwise_constraint=keras.constraints.NonNeg()
    )(base_model.output)

    flat = keras.layers.Flatten()(depth)
    preds = keras.layers.Dense(
        6,
        activation='softmax',
        kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.01),
        bias_initializer=keras.initializers.Zeros()
    )(flat)

    model = keras.Model(inputs=base_model.input, outputs=preds)

for layer in model.layers:
    layer.trainable = True

model.summary()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Number of devices: 1


Model: "functional"

In [21]:
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

with strategy.scope():
    input_tensor = keras.Input(shape=(224, 224, 3))
    base_model = keras.applications.DenseNet169(input_tensor=input_tensor, weights=None, include_top=False)
    depth = keras.layers.DepthwiseConv2D(
        tuple(base_model.output.shape[1:3]),
        depthwise_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.01),
        bias_initializer=keras.initializers.Zeros(),
        depthwise_constraint=keras.constraints.NonNeg()
    )(base_model.output)

    flat = keras.layers.Flatten()(depth)
    preds = keras.layers.Dense(
        6,
        activation='softmax',
        kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.01),
        bias_initializer=keras.initializers.Zeros()
    )(flat)

    model = keras.Model(inputs=base_model.input, outputs=preds)
    # ModelCheckpoint
    filepath = "models/{}-{{epoch:02d}}-{{val_accuracy:.4f}}.keras".format(name)
    checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=False, mode='max')  # Create checkpoint to save the best validation accuracy
    callbacks_list = [checkpoint]

    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=0.045,
        decay_steps=2 * int(train_img_num / batch_size),
        decay_rate=0.94,
        staircase=True
    )
    optimizer = keras.optimizers.SGD(momentum=0.9, learning_rate=lr_schedule)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    # Start training
    hist = model.fit(
        train_generator,
        epochs=80,
        validation_data=validation_generator,
        shuffle=True,
        callbacks=callbacks_list
    )

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Number of devices: 1
Epoch 1/80


2024-07-30 13:40:16.771053: W tensorflow/core/framework/dataset.cc:959] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
/opt/homebrew/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 1/86 ━━━━━━━━━━━━━━━━━━━━ 1:00:59 43s/step - accuracy: 0.1714 - loss: 1.7905

KeyboardInterrupt: 

In [13]:
# Get the accuracy and validation accuracy from the history
acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']

# Get the number of epochs
epochs = range(1, len(acc) + 1)

# Plot training and validation accuracy
plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()
# Get the accuracy and validation accuracy from the history
acc = hist.history['loss']
val_acc = hist.history['val_loss']

# Get the number of epochs
epochs = range(1, len(acc) + 1)

# Plot training and validation accuracy
plt.plot(epochs, acc, 'b', label='Training loss')
plt.plot(epochs, val_acc, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('loss')
plt.legend()

plt.show()

NameError: name 'hist' is not defined